In [ ]:
# Python 버전 확인
!python --version

# 설치된 패키지 목록을 requirements.txt로 저장
!pip list --format=freeze > requirements.txt

# requirements.txt 내용 확인
!cat requirements.txt

Python 3.10.12
absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.19
albumentations==1.4.20
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.20.0
astropy==6.1.4
astropy-iers-data==0.2024.10.21.0.33.21
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.24.0
bigquery-magics==0.4.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
Bottleneck==1.4.2
bqplot==0.12.43
branca==0.8.0
CacheControl==0.14.0
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.0
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.30.5
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.

# Mission 2. 패션 스타일 선호 여부 예측

## 2-1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import re
import time

In [ ]:
# 데이터 경로 설정
training_image_path = "/content/drive/MyDrive/final/dataset/training_image"
validation_image_path = "/content/drive/MyDrive/final/dataset/validation_image"
training_label_path = "/content/drive/MyDrive/final/dataset/training_label"
validation_label_path = "/content/drive/MyDrive/final/dataset/validation_label"

In [ ]:
# 데이터 수 확인 (배치 처리, 대기 시간 추가)
def count_files_batch(folder_path, extension=None, batch_size=5000, sleep_time=0.1):
    count = 0
    try:
        with os.scandir(folder_path) as entries:
            file_names = [entry.name for entry in entries if entry.is_file() and (extension is None or entry.name.endswith(extension))]

        for i in range(0, len(file_names), batch_size):
            batch_files = file_names[i:i + batch_size]
            count += len(batch_files)
            time.sleep(sleep_time)  # 대기 시간 추가
    except OSError as e:
        print(f"Error reading folder {folder_path}: {e}")
    return count

num_training_images = count_files_batch(training_image_path)
num_training_labels = count_files_batch(training_label_path)
num_validation_images = count_files_batch(validation_image_path)
num_validation_labels = count_files_batch(validation_label_path)

print(f"Number of training images: {num_training_images}")
print(f"Number of training labels: {num_training_labels}")
print(f"Number of validation images: {num_validation_images}")
print(f"Number of validation labels: {num_validation_labels}")

Number of training images: 4070
Number of training labels: 211345
Number of validation images: 951
Number of validation labels: 36382


In [ ]:
# 이미지ID를 추출하는 함수 (정규식 사용)
def extract_image_id(filename):
    # 정규식 패턴 정의
    pattern = r"^(W|T)_(\d+)"

    # 정규식 검색
    match = re.match(pattern, filename)

    if match:
        return f"{match.group(1)}_{match.group(2)}"  # "W" 또는 "T"와 숫자 ID 부분 추출
    else:
        return None  # 매칭되지 않는 경우 None 반환

In [ ]:
# 유효한 라벨링 데이터 추출 함수 (배치 처리)
def get_valid_labels(image_path, label_path, batch_size=5000):
    valid_labels = []
    try:
        # 이미지 파일 리스트
        image_files = []
        with os.scandir(image_path) as entries:
            for entry in entries:
                if entry.is_file() and entry.name.endswith('.jpg'):
                    image_files.append(entry.name)

        valid_image_ids = {extract_image_id(f) for f in image_files}

        # 라벨 파일 리스트 배치로 처리
        label_files = []
        with os.scandir(label_path) as entries:
            for entry in entries:
                if entry.is_file():
                    label_files.append(entry.name)

        for i in range(0, len(label_files), batch_size):
            batch_files = label_files[i:i + batch_size]
            valid_labels.extend([f for f in batch_files if extract_image_id(f) in valid_image_ids])
    except OSError as e:
        print(f"Error reading folder {label_path}: {e}")

    return valid_labels

In [ ]:
# 성별과 스타일을 추출하고 설문ID 수 및 이미지 수를 계산하는 함수
def calculate_statistics(label_files, image_path):
    stats = {}
    image_files = os.listdir(image_path)

    # 정규식 패턴 정의 (파일명에서 성별, 스타일, 설문ID를 추출)
    pattern = r"^(W|T)_(\d+)_(\d+)_(\w+)_(W|M)_(\d+)\.json"

    for file in label_files:
        match = re.match(pattern, file)

        if match:
            style = match.group(4)  # 스타일
            gender = match.group(5)  # 성별 (W -> 여성, M -> 남성으로 변환)
            gender = '여성' if gender == 'W' else '남성'
            survey_id = match.group(6)  # 설문ID
            image_id = extract_image_id(file)  # 이미지ID

            # 성별 & 스타일 키 생성
            key = (gender, style)

            if key not in stats:
                stats[key] = {'survey_ids': set(), 'image_count': 0}

            # 설문ID 추가
            stats[key]['survey_ids'].add(survey_id)

            # 해당 이미지ID가 이미지 데이터에 존재하면 이미지 수 증가
            if any(image_id in img_file for img_file in image_files):
                stats[key]['image_count'] += 1

    # 설문ID 수로 변환
    stats_count = {k: {'설문ID 수': len(v['survey_ids']), '이미지 수': v['image_count']} for k, v in stats.items()}

    return stats_count

In [ ]:
# 통계표를 데이터프레임으로 변환하는 함수 (성별과 스타일로 정렬)
def create_dataframe(stats):
    df = pd.DataFrame(list(stats.items()), columns=["성별&스타일", "통계"])
    df[['성별', '스타일']] = pd.DataFrame(df['성별&스타일'].tolist(), index=df.index)
    df = pd.concat([df.drop(columns='통계'), df['통계'].apply(pd.Series)], axis=1)

    # 성별&스타일, 설문ID 수 제외
    df = df[['성별', '스타일', '이미지 수']]

    # 성별(여성 -> 남성) 순으로, 이미지 수(높은 수 -> 낮은 수) 순으로 정렬
    df = df.sort_values(by=['성별', '이미지 수'], ascending=[False, False])

    # 인덱스 초기화하여 순서 번호 재정렬
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
# 유효한 라벨링 데이터 추출
valid_train_labels = get_valid_labels(training_image_path, training_label_path)
valid_val_labels = get_valid_labels(validation_image_path, validation_label_path)

# 성별&스타일별 설문ID 및 이미지 수 통계 계산
train_stats = calculate_statistics(valid_train_labels, training_image_path)
val_stats = calculate_statistics(valid_val_labels, validation_image_path)

# 통계표 작성
train_df = create_dataframe(train_stats)
val_df = create_dataframe(val_stats)

In [ ]:
print("Training 데이터 통계표:")
train_df

Training 데이터 통계표:


,성별,스타일,이미지 수
0,여성,sportivecasual,815
1,여성,feminine,719
2,여성,minimal,643
3,여성,powersuit,531
4,여성,bodyconscious,455
5,여성,athleisure,378
6,여성,classic,360
7,여성,hippie,308
8,여성,oriental,282
9,여성,normcore,278


In [ ]:
print("Validation 데이터 통계표:")
val_df

Validation 데이터 통계표:


,성별,스타일,이미지 수
0,여성,sportivecasual,254
1,여성,feminine,208
2,여성,minimal,164
3,여성,powersuit,151
4,여성,bodyconscious,111
5,여성,classic,102
6,여성,athleisure,80
7,여성,space,70
8,여성,oriental,67
9,여성,ecology,65


## 2-2.

In [ ]:
import json
import os
import pandas as pd
from IPython.display import display

In [ ]:
# 경로 설정
train_label_path = '/content/drive/MyDrive/final/dataset/training_label'
val_label_path = '/content/drive/MyDrive/final/dataset/validation_label'
train_image_path = '/content/drive/MyDrive/final/dataset/training_image'
val_image_path = '/content/drive/MyDrive/final/dataset/validation_image'

In [ ]:
# JSON 데이터를 읽고 응답자 ID와 스타일 선호 정보를 추출하는 함수 (배치 처리)
def extract_survey_info(label_files, label_path, batch_size=10000):
    survey_info = {}
    try:
        for i in range(0, len(label_files), batch_size):
            batch_files = label_files[i:i + batch_size]
            for file in batch_files:
                file_path = os.path.join(label_path, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # 응답자 ID (user > R_id)
                respondent_id = data['user']['R_id']

                # 이미지 파일명 (item > imgName)
                image_file = data['item']['imgName']

                # 스타일 선호 여부 (item > survey > Q5)
                style_preference = data['item']['survey']['Q5']

                if respondent_id not in survey_info:
                    survey_info[respondent_id] = {'training': [], 'validation': []}

                # 파일이 어느 폴더에 속하는지에 따라 Training 또는 Validation으로 구분
                if label_path == training_label_path:
                    survey_info[respondent_id]['training'].append((image_file, style_preference))
                else:
                    survey_info[respondent_id]['validation'].append((image_file, style_preference))
    except OSError as e:
        print(f"Error reading folder {label_path}: {e}")

    return survey_info

In [ ]:
# 유효한 라벨링 데이터 추출 (2-1에서 구한 유효한 데이터 사용)
valid_train_labels = get_valid_labels(train_image_path, train_label_path)
valid_val_labels = get_valid_labels(val_image_path, val_label_path)

# 유효한 라벨링 데이터에서 응답자별 스타일 선호 정보 추출
train_survey_info = extract_survey_info(valid_train_labels, train_label_path)
val_survey_info = extract_survey_info(valid_val_labels, val_label_path)

In [ ]:
# Training과 Validation 모두에 존재하는 응답자 ID만 선택
common_respondent_ids = set(train_survey_info.keys()) & set(val_survey_info.keys())

# 공통 응답자 정보만 통합
common_survey_info = {respondent_id: {
    'training': train_survey_info[respondent_id]['training'],
    'validation': val_survey_info[respondent_id]['validation']
} for respondent_id in common_respondent_ids}

# 응답자별 응답 수 계산 (Training + Validation 응답 수 합산)
respondent_response_count = []

for respondent_id, info in common_survey_info.items():
    train_responses = len(info['training'])  # Training 응답 수
    val_responses = len(info['validation'])  # Validation 응답 수
    total_responses = train_responses + val_responses  # 총 응답 수
    respondent_response_count.append({
        '응답자ID': respondent_id,
        '총 응답 수': total_responses
    })

# 데이터프레임으로 변환
response_df = pd.DataFrame(respondent_response_count)

# 응답 수 기준으로 상위 100명 선택
top_100_respondents = response_df.nlargest(100, '총 응답 수')['응답자ID']

# 상위 100명의 응답자 정보만 추출
limited_survey_info = {respondent_id: common_survey_info[respondent_id] for respondent_id in top_100_respondents}

In [ ]:
# 100명의 응답자 정보를 통합한 JSON으로부터 데이터프레임 생성
data = []

for respondent_id, info in limited_survey_info.items():
    train_preferences = info['training']
    val_preferences = info['validation']

    # Training 데이터에서 파일명을 줄바꿈으로 연결
    train_preferred_files = "\n".join([file for file, pref in train_preferences if pref == 2])
    train_not_preferred_files = "\n".join([file for file, pref in train_preferences if pref == 1])

    # Validation 데이터에서 파일명을 줄바꿈으로 연결
    val_preferred_files = "\n".join([file for file, pref in val_preferences if pref == 2])
    val_not_preferred_files = "\n".join([file for file, pref in val_preferences if pref == 1])

    data.append({
        '응답자ID': respondent_id,
        'Training 스타일 선호': train_preferred_files,
        'Training 스타일 비선호': train_not_preferred_files,
        'Validation 스타일 선호': val_preferred_files,
        'Validation 스타일 비선호': val_not_preferred_files
    })

# 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터프레임 스타일링: 줄바꿈을 적용하여 한 줄에 하나씩 출력되도록 설정
df_styled = df.style.set_properties(**{
    'Training 스타일 선호': 'white-space: pre-wrap;',
    'Training 스타일 비선호': 'white-space: pre-wrap;',
    'Validation 스타일 선호': 'white-space: pre-wrap;',
    'Validation 스타일 비선호': 'white-space: pre-wrap;'
})

# 데이터프레임 출력
display(df_styled)

,응답자ID,Training 스타일 선호,Training 스타일 비선호,Validation 스타일 선호,Validation 스타일 비선호
0,64747,W_48628_19_genderless_W.jpg W_46907_80_powersuit_W.jpg W_44330_10_sportivecasual_W.jpg W_39725_19_normcore_W.jpg W_38421_10_athleisure_W.jpg W_38629_80_powersuit_W.jpg W_37160_70_punk_W.jpg W_35674_60_minimal_W.jpg W_34636_00_oriental_W.jpg W_31416_70_hippie_W.jpg W_30399_19_genderless_W.jpg W_30434_60_minimal_W.jpg W_30454_60_minimal_W.jpg W_29783_10_sportivecasual_W.jpg W_22510_80_powersuit_W.jpg W_22057_19_genderless_W.jpg W_21223_80_powersuit_W.jpg W_21483_19_genderless_W.jpg W_20598_70_military_W.jpg W_14901_90_kitsch_W.jpg W_07894_00_cityglam_W.jpg W_05628_00_cityglam_W.jpg W_04972_90_kitsch_W.jpg W_03194_50_classic_W.jpg,W_48378_90_grunge_W.jpg W_47169_70_hippie_W.jpg W_44386_80_powersuit_W.jpg W_42595_60_popart_W.jpg W_41633_60_space_W.jpg W_40876_70_punk_W.jpg W_40690_80_bodyconscious_W.jpg W_38771_10_sportivecasual_W.jpg W_37025_19_lounge_W.jpg W_36644_00_oriental_W.jpg W_34573_10_sportivecasual_W.jpg W_34024_10_sportivecasual_W.jpg W_33026_90_hiphop_W.jpg W_27828_60_minimal_W.jpg W_18951_50_feminine_W.jpg W_13904_50_feminine_W.jpg W_14102_50_feminine_W.jpg W_11610_90_grunge_W.jpg W_03643_00_cityglam_W.jpg W_02498_50_feminine_W.jpg W_02247_50_classic_W.jpg,W_46907_80_powersuit_W.jpg W_44330_10_sportivecasual_W.jpg W_38588_19_genderless_W.jpg W_39164_00_oriental_W.jpg W_37491_70_military_W.jpg W_30988_90_kitsch_W.jpg W_22510_80_powersuit_W.jpg W_20598_70_military_W.jpg W_05628_00_cityglam_W.jpg,W_47169_70_hippie_W.jpg W_34024_10_sportivecasual_W.jpg W_27828_60_minimal_W.jpg W_14102_50_feminine_W.jpg W_11610_90_grunge_W.jpg W_02498_50_feminine_W.jpg
1,63405,W_17849_00_metrosexual_M.jpg W_17909_60_mods_M.jpg W_17510_19_normcore_M.jpg W_16084_80_bold_M.jpg W_15400_60_mods_M.jpg W_15294_50_ivy_M.jpg W_12814_19_normcore_M.jpg W_12551_19_normcore_M.jpg W_07198_10_sportivecasual_M.jpg W_06785_50_ivy_M.jpg W_06812_50_ivy_M.jpg W_07021_90_hiphop_M.jpg W_06682_00_metrosexual_M.jpg W_04684_90_hiphop_M.jpg W_04522_50_ivy_M.jpg W_04206_60_mods_M.jpg W_02890_19_normcore_M.jpg W_02931_00_metrosexual_M.jpg W_01853_60_mods_M.jpg W_01670_10_sportivecasual_M.jpg W_00555_50_ivy_M.jpg W_00023_60_mods_M.jpg,W_17599_19_normcore_M.jpg W_17219_70_hippie_M.jpg W_17443_90_hiphop_M.jpg W_17454_80_bold_M.jpg W_16736_00_metrosexual_M.jpg W_16755_00_metrosexual_M.jpg W_16919_19_normcore_M.jpg W_17108_19_normcore_M.jpg W_16501_70_hippie_M.jpg W_15782_70_hippie_M.jpg W_15471_10_sportivecasual_M.jpg W_15472_70_hippie_M.jpg W_15517_70_hippie_M.jpg W_15134_80_bold_M.jpg W_15140_80_bold_M.jpg W_12904_50_ivy_M.jpg W_12383_80_bold_M.jpg W_12443_90_hiphop_M.jpg W_12456_10_sportivecasual_M.jpg W_06691_10_sportivecasual_M.jpg W_02714_00_metrosexual_M.jpg W_00530_10_sportivecasual_M.jpg,W_15294_50_ivy_M.jpg W_06860_19_normcore_M.jpg W_04522_50_ivy_M.jpg W_04684_90_hiphop_M.jpg W_01853_60_mods_M.jpg W_02677_60_mods_M.jpg W_02879_90_hiphop_M.jpg,W_16501_70_hippie_M.jpg W_16755_00_metrosexual_M.jpg W_17443_90_hiphop_M.jpg W_15140_80_bold_M.jpg W_12304_80_bold_M.jpg W_12904_50_ivy_M.jpg W_07187_70_hippie_M.jpg
2,64346,W_49510_00_metrosexual_M.jpg W_29918_19_normcore_M.jpg W_29990_90_hiphop_M.jpg W_30040_60_mods_M.jpg W_28693_10_sportivecasual_M.jpg W_26175_50_ivy_M.jpg W_24874_80_bold_M.jpg W_24977_70_hippie_M.jpg W_24268_00_metrosexual_M.jpg W_23905_60_mods_M.jpg W_23920_19_normcore_M.jpg W_23922_50_ivy_M.jpg W_24103_50_ivy_M.jpg W_21401_00_metrosexual_M.jpg W_16777_60_mods_M.jpg W_16233_80_bold_M.jpg W_12527_70_hippie_M.jpg W_12155_80_bold_M.jpg W_09856_10_sportivecasual_M.jpg W_09154_50_ivy_M.jpg W_09233_60_mods_M.jpg W_09239_19_normcore_M.jpg W_09289_19_normcore_M.jpg W_02754_70_hippie_M.jpg W_00856_10_sportivecasual_M.jpg W_00047_70_hippie_M.jpg,W_33015_19_normcore_M.jpg W_28908_90_hiphop_M.jpg W_25543_60_mods_M.jpg W_25721_90_hiphop_M.jpg W_25828_10_sportivecasual_M.jpg W_25391_00_metrosexual_M.jpg W_24832_80_bold_M.jpg W_24838_70_hippie_M.jpg W_24931_50_ivy_M.jpg W_25065_10

In [ ]:
# CSV 파일로 저장
output_csv_path = '/content/drive/MyDrive/final/스타일선호도.csv'
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')